In [ ]:
import pandas as pd
import altair as alt
import json, os, glob, pdb

import ROOT

pd.options.mode.chained_assignment = None


In [ ]:
df = pd.read_csv("Mergedcsvfile_perLS.csv")
sigma_fid = 575.180476828
df['zLumi'] = df['zDel'] / sigma_fid
df['zLumi_mc'] = df['zDel_mc'] / sigma_fid

df['ctime'] = df['time'].apply(lambda x: ROOT.TDatime(x).AsSQLString())

print(df.keys())
print(len(df))


#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df.query('run == 297219')[['time','dtime','dtime2','ls']])

In [ ]:
brush = alt.selection_interval()

selection = alt.selection_multi(fields=['run'])


color1 = alt.condition(selection & brush,
                      alt.Color('run:N',
                      legend=None),
                      alt.value('lightgray')
                     )

color2 = alt.condition(selection,
                      alt.Color('run:N',
                      legend=None),
                      alt.value('lightgray')
                     )

tooltips = [
    'run',
    'ls',
    'zYield',
]


In [ ]:
charts = []

for fill, df_fill in df.groupby('fill'):
    
    df_fill = df_fill.sort_values(by=['time'])
    
    df_fill['Z lumi (w/o correction)'] = df_fill['zLumi'].cumsum()
    df_fill['Z lumi'] = df_fill['zLumi_mc'].cumsum()
    df_fill['reference lumi'] = df_fill['recorded(/pb)'].cumsum()
    
    legend = alt.Chart(df_fill).mark_circle(size=60).encode(
        y=alt.Y('run:N', axis=alt.Axis(orient='right'), title='run'),
        color=color2
    ).add_selection(
         selection
    )

    chart_base = alt.Chart(df_fill).encode(
        x=alt.X('ctime:T', scale=alt.Scale(zero=False), title='time'),
        color=color2,
        tooltip=tooltips,
    ).properties(
        width=500,
        height=250
    )

    # instantanious luminosity
    chart_inst_zLumi = chart_base.mark_circle(size=60).encode(
        y=alt.Y('zLumi_mc', scale=alt.Scale(zero=False)),
    )

    # reference lumi as line
    chart_inst_refLumi = chart_base.mark_line().encode(
        opacity=alt.value(0.5),
        color=alt.value('black'),
        y=alt.Y('recorded(/pb)', scale=alt.Scale(zero=False), title='instantanious luminosity in 1/pb'),
    )

    chart_inst = alt.layer(chart_inst_zLumi,chart_inst_refLumi).add_selection(
         selection
    ).transform_filter(
         selection 
    ).interactive()

    # integrated luminosity
    chart_int_base = chart_base.transform_fold(
        ['Z lumi', 'Z lumi (w/o correction)', 'reference lumi'],
    )

    chart_int = chart_int_base.mark_line().encode(
        x=alt.X('ctime:T', scale=alt.Scale(zero=False), title='time'),
        y=alt.Y('value:Q', scale=alt.Scale(zero=False), title='integrated luminosity in 1/pb'),
        strokeDash=alt.X(
            'key:N',
            legend=alt.Legend(orient='none',legendX=30,legendY=330, fillColor='white', title='source'),
        ),
    ).add_selection(
         selection
    ).transform_filter(
         selection 
    ).interactive()

    chart_inst = alt.hconcat(chart_inst, legend)
    chart_combined = alt.vconcat(chart_inst, chart_int)
    
    chart_combined.save(f'Charts_Fill/chart_fill{fill}.html', embed_options={'renderer':'svg'})
        
    #chart_to_html(chart_combined, "Charts_Fill/luminosity_fill_"+str(fill), _data=f"Data/data_fill{fill}.csv")
    charts.append(chart_combined)
    

In [ ]:
charts[0]